In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import numpy as np
import pandas as pd
import re
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from textblob import Word, TextBlob
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
fixed_skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [3]:
train = fetch_20newsgroups()
test = fetch_20newsgroups(subset="test")
X_train, y_train = train['data'], train['target']
X_test, y_test = test['data'], test['target']

## Preprocessing and regex features:

In [4]:
clean_sentences = lambda text: ''.join(filter(lambda x: x.isalpha() or x in [' ', '\n', '.', ',', ':', ',', '!', '?'] or x.isdigit(), text))
get_sentences = lambda text: re.findall(r'[A-Z][^\.]*[\.|\?|\!][$|\s]', text)
get_weird_sentences = lambda text: re.findall(r'[\s|^|\.][A-Z][^\.]*[\.|\?|\!]', text)

In [5]:
def remove_digits(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    return text

In [6]:
def split_specific(data, topics):
    split_txts = []
    for txt in data:
        split_txt = {}
        for topic in topics:
            
            if topic+':' not in txt:
                split_txt[topic] = ''
            else:
                s = txt.split(topic + ':')[1]
                next_topic = re.findall('[a-z\-A-Z]+:',s)
                if next_topic:
#                     s = s.split(re.findall('[a-z\-A-Z]+:',s)[0])[0]
                    s = s.split(re.findall('[a-z\-A-Z]+:|$',s)[0])[0]
                split_txt[topic] = s
                txt = txt.replace(s, ' ').replace(topic+':', ' ')         
        split_txt['the_rest'] = txt
        split_txts += [split_txt]
        
    return split_txts

In [7]:
letters_only = lambda txt: ''.join(filter(lambda c: c.isalpha(), txt)).lower()

In [8]:
uppercase_words = lambda text: len(re.findall('[A-Z]{2,}', text))

In [31]:
def prepare_regex(X):
    df = pd.DataFrame(X, columns=['text'])
    df['text'] = df['text'].apply(clean_sentences)
    df['question_marks_cnt'] = df['text'].str.count('\?')
    df['exclamation_marks_cnt'] = df['text'].str.count('\!')
    df['uppercase_words_count'] = df['text'].apply(uppercase_words)
    df_ = pd.DataFrame(split_specific(df['text'], ['From']))
    df_['From'] = df_['From'].apply(lambda f: ' '.join(f.strip().split(' ')[0].split('.')))
    df_['sentences'] = df_.the_rest.apply(get_sentences)
    df_['sentences_count'] = df_['sentences'].apply(len)
    df_.loc[df_.sentences_count == 0, 'the_rest'] = df_.loc[df_.sentences_count == 0, 'the_rest'].apply(get_weird_sentences)
    df_['sentences_count'] = df_['sentences'].apply(len)
    df_.loc[df_.sentences_count == 0, 'sentences'] = [['']]*df_.loc[df_.sentences_count == 0, 'sentences'].shape[0]
    df_['median_sentence_len'] = df_['sentences'].apply(lambda arr: np.median(np.array([len(item) for item in arr]), axis=0))
    df_['polarity_1_sntc'] = df_['sentences'].apply(lambda s: TextBlob(s[0]).sentiment.polarity if len(s) > 0 else np.nan)
    df_['subjectivity_1_sntc'] = df_['sentences'].apply(lambda s: TextBlob(s[0]).sentiment.subjectivity  if len(s) > 0 else np.nan)
    df_['polarity_last_sntc'] = df_['sentences'].apply(lambda s: TextBlob(s[-1]).sentiment.polarity  if len(s) > 0 else np.nan)
    df_['subjectivity_last_sntc'] = df_['sentences'].apply(lambda s: TextBlob(s[-1]).sentiment.subjectivity  if len(s) > 0 else np.nan)
    df_.drop(['sentences', 'the_rest'], axis=1, inplace=True)
    df = pd.concat([df, df_], axis=1)
    return df

In [13]:
df.head(2)

,text,question_marks_cnt,exclamation_marks_cnt,uppercase_words_count,From,sentences_count,median_sentence_len,polarity_1_sntc,subjectivity_1_sntc,polarity_last_sntc,subjectivity_last_sntc
0,From: lerxstwam.umd.edu wheres my thing\nSubje...,1,1,2,lerxstwam umd edu,8,49.0,-0.208333,0.333333,0.0,0.0
1,From: guykuocarson.u.washington.edu Guy Kuo\nS...,0,0,7,guykuocarson u washington edu,4,94.0,0.750000,0.950000,0.2,0.2


## TF-IDF Vectorizer

In [80]:
df = prepare_regex(X_train)

C:\Users\ASUS\machine_learning_projector\venv\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [81]:
vectorizer = TfidfVectorizer(
                    analyzer='word',
                    ngram_range=(1,2),
                    stop_words='english',
                    vocabulary=None,
                    max_df=0.4, #0.8 
                    min_df = 30, #50
                    max_features=None,
                    smooth_idf=False,
                    norm='l2',
                    preprocessor=remove_digits)
bag = vectorizer.fit_transform(df['text'])
words = pd.DataFrame(bag.astype(np.float32).todense(), columns = vectorizer.get_feature_names_out())

In [82]:
vectorizer = TfidfVectorizer(
                    analyzer='char_wb',
                    ngram_range=(4,9),
                    vocabulary=None,
                    max_df=400, #0.8 
                    min_df = 200, #50
                    max_features=None,
                    smooth_idf=False,
                    norm='l2',
                    preprocessor=letters_only)
bag = vectorizer.fit_transform(df['text'])
chars = pd.DataFrame(bag.astype(np.float32).todense(), columns = vectorizer.get_feature_names_out())

In [83]:
final = pd.concat([words, chars, df.drop(['text', 'From'], axis=1)], axis=1)
final.drop(final.columns[final.columns.duplicated()], axis=1, inplace=True)

In [73]:
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
final_ = StandardScaler().fit_transform(final_)
best_final_ = pd.DataFrame(final_, columns=final.columns)

**Best model minus regex**

In [84]:
final = pd.concat([words, chars], axis=1)
final.drop(final.columns[final.columns.duplicated()], axis=1, inplace=True)
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
best_final_ = pd.DataFrame(final_, columns=final.columns)

In [93]:
reg = LogisticRegression(penalty='l1',
                         C=0.75,
                         multi_class='ovr', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42,
                         solver='liblinear')
logreg_bot_scaled = cross_validate(reg, best_final_, y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

In [92]:
pd.DataFrame(logreg_bot_scaled).mean(axis=0) #2

fit_time       6.489713
score_time     0.901423
test_score     0.770461
train_score    0.811605
dtype: float64

In [94]:
reg.fit(final_, y_train)
best_weights = list(zip(np.abs(reg.coef_).sum(axis=0), best_final_.columns))
best_weights.sort(key = lambda x: x[0], reverse=True)

C:\Users\ASUS\machine_learning_projector\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(


In [109]:
selected_columns = [x[1] for x in best_weights[:3000]]

In [115]:
reg = LogisticRegression(penalty='l2',
                         C=0.0000005,
                         multi_class='multinomial', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42)
logreg_best = cross_validate(reg, best_final_[selected_columns], y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ..................., score=(train=0.820, test=0.793) total time=   1.8s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] END ..................., score=(train=0.830, test=0.793) total time=   1.9s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.2s remaining:    0.0s


[CV] END ..................., score=(train=0.827, test=0.787) total time=   2.2s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    6.8s finished


In [116]:
reg.fit(best_final_[selected_columns], y_train)
y_preds = reg.predict(test_prep[selected_columns])
accuracy_score(y_preds, y_test)

0.7489378651088688

## best model


**Selecting features**

In [122]:
final = pd.concat([words, chars, df.drop(['text', 'From'], axis=1)], axis=1)
final.drop(final.columns[final.columns.duplicated()], axis=1, inplace=True)
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
final_ = StandardScaler().fit_transform(final_)
best_final_ = pd.DataFrame(final_, columns=final.columns)

In [123]:
reg = LogisticRegression(penalty='l1',
                         C=0.1,
                         multi_class='ovr', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42,
                         solver='liblinear')
reg.fit(final_, y_train)
best_weights = list(zip(np.abs(reg.coef_).sum(axis=0), best_final_.columns))
best_weights.sort(key = lambda x: x[0], reverse=True)

C:\Users\ASUS\machine_learning_projector\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(


In [124]:
regex_cols = df.drop(['text', 'From'], axis=1).columns.tolist()

In [125]:
best_weights[:5]

[(4.125387278458404, 'sentences_count'),
 (1.4124423860737758, 'windows'),
 (1.2282296483211368, 'sale'),
 (1.1711097826738268, 'dod'),
 (0.9782444557775232, 'car')]

In [126]:
list(filter(lambda x: x[1] in regex_cols, best_weights))

[(4.125387278458404, 'sentences_count'),
 (0.093578618837925, 'uppercase_words_count'),
 (0.059680283522817706, 'question_marks_cnt'),
 (0.02386087730195255, 'subjectivity_1_sntc'),
 (0.0017738851442983998, 'polarity_last_sntc'),
 (0.0, 'exclamation_marks_cnt'),
 (0.0, 'median_sentence_len'),
 (0.0, 'polarity_1_sntc'),
 (0.0, 'subjectivity_last_sntc')]

All but one regex features are useless, but that one, 'sentences_count', is the most important for logreg

**Logistic Regression with 5000 most important features**

In [167]:
selected_columns = [x[1] for x in best_weights[:5000]]

In [168]:
reg = LogisticRegression(penalty='l2',
                         C=0.01,
                         multi_class='multinomial', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42)
logistic_reg_results_tfidf = cross_validate(reg, best_final_[selected_columns], y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ..................., score=(train=1.000, test=0.931) total time=   9.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s remaining:    0.0s


[CV] END ..................., score=(train=1.000, test=0.934) total time=   9.2s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   18.5s remaining:    0.0s


[CV] END ..................., score=(train=1.000, test=0.926) total time=   9.7s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   28.3s finished


In [169]:
pd.DataFrame(logistic_reg_results_tfidf).mean(axis=0)

fit_time       9.294888
score_time     0.062602
test_score     0.930440
train_score    0.999779
dtype: float64

**Best model interpretation and test score**

In [170]:
reg = LogisticRegression(penalty='l2',
                         C=0.01,
                         multi_class='multinomial', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42)
reg.fit(best_final_[selected_columns], y_train)

LogisticRegression(C=0.01, class_weight='balanced', multi_class='multinomial',
                   n_jobs=4, random_state=42)

In [ ]:
df = prepare_regex(X_test)

In [139]:
words_vocabulary = list(words.columns)
chars_vocabulary = list(chars.columns)

In [140]:
vectorizer = TfidfVectorizer(
                    analyzer='word',
                    ngram_range=(1,2),
                    stop_words='english',
                    vocabulary=words_vocabulary,
                    max_df=0.4, #0.8 
                    min_df = 30, #50
                    max_features=None,
                    smooth_idf=True,
                    norm='l2',
                    preprocessor=remove_digits)
bag = vectorizer.fit_transform(df['text'])
words = pd.DataFrame(bag.astype(np.float32).todense(), columns = vectorizer.get_feature_names_out())

In [141]:
vectorizer = TfidfVectorizer(
                    analyzer='char_wb',
                    ngram_range=(4,9),
                    vocabulary=chars_vocabulary,
                    max_df=400, #0.8 
                    min_df = 200, #50
                    max_features=None,
                    smooth_idf=True,
                    norm='l2',
                    preprocessor=letters_only)
bag = vectorizer.fit_transform(df['text'])
chars = pd.DataFrame(bag.astype(np.float32).todense(), columns = vectorizer.get_feature_names_out())

In [142]:
final = pd.concat([words, chars, df.drop(['text', 'From'], axis=1)], axis=1)
final.drop(final.columns[final.columns.duplicated()], axis=1, inplace=True)

In [143]:
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
final_ = StandardScaler().fit_transform(final_)
test_prep = pd.DataFrame(final_, columns=final.columns)

In [171]:
y_pred = reg.predict(test_prep[selected_columns])

In [172]:
accuracy_score(y_test, y_pred)

0.7955390334572491

3000 features: 0.78

In [173]:
import eli5

In [180]:
eli5.show_weights(reg, feature_names = selected_columns, target_names=train['target_names'])

In [190]:
best_final_.loc[best_final_['pittedu'] > 0].head(3)

,aa,aaa,aaron,ab,abandoned,abc,ability,able,able use,abortion,...,zine,question_marks_cnt,exclamation_marks_cnt,uppercase_words_count,sentences_count,median_sentence_len,polarity_1_sntc,subjectivity_1_sntc,polarity_last_sntc,subjectivity_last_sntc
53,-0.0613,-0.051056,-0.071476,-0.078557,-0.041258,-0.050063,-0.114927,-0.239743,-0.050097,-0.052426,...,-0.120898,-0.093352,-0.086395,-0.092054,-0.037810,-0.461526,0.123626,-1.296358,-0.211684,0.698994
177,-0.0613,-0.051056,-0.071476,-0.078557,-0.041258,-0.050063,-0.114927,-0.239743,-0.050097,-0.052426,...,-0.120898,-0.049794,-0.086395,-0.080659,-0.221437,-0.110560,-0.720481,0.160459,0.139711,-0.435880
251,-0.0613,-0.051056,-0.071476,-0.078557,-0.041258,-0.050063,-0.114927,-0.239743,-0.050097,-0.052426,...,-0.120898,-0.093352,-0.086395,-0.074961,0.145817,-0.126884,-0.298427,0.524664,0.420827,0.772212


The names of the classes not once are the most weighted words in a bag, so maybe it would be a good idea to make a feature out of a class name. The model also values 'edu keith' in a class named 'atheism'. That probably indicates that trainset has a lot of emails from 'edu keith' featuring that theme. It's bad for generalization, because Keith is a person, so maybe it would be a good idea to extract email adresses from the set. Also, 'caltech edu' refers to 'keith edu' as that is one man's email. The reason I left the emails was to spot organizations that are most active in a particular field. Like 'pitt' which is a domen for Univercity of Pittsburg Computer Science, but that adress is valued high in 'medical' class, which doesn't make sense logically. In other cases, it's pretty self-explanatory and seems reasonable: class 'baseball' values 'sox' and 'stadium', 'guns' - 'cnn' and 'fbi', 'mideast' - 'israel'.

In [193]:
X_train[177][:200]

'From: geb@cs.pitt.edu (Gordon Banks)\nSubject: Re: tuberculosis\nReply-To: geb@cs.pitt.edu (Gordon Banks)\nOrganization: Univ. of Pittsburgh Computer Science\nLines: 20\n\nIn article <1993Mar25.020646.852@n'

In [195]:
train['target_names'][y_train[177]]

'sci.med'

**Logistic Regression, not scaled**

In [33]:
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
# final_ = StandardScaler().fit_transform(final_)
final_ = pd.DataFrame(final_, columns=final.columns)

In [37]:
reg = LogisticRegression(penalty='l2',
                         C=0.01,
                         multi_class='multinomial', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42)
logreg_bot_scaled = cross_validate(reg, final_[selected_columns], y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ..................., score=(train=0.174, test=0.169) total time=  15.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.1s remaining:    0.0s


[CV] END ..................., score=(train=0.151, test=0.152) total time=  17.4s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   33.8s remaining:    0.0s


[CV] END ..................., score=(train=0.158, test=0.160) total time=  17.8s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   51.8s finished


In [38]:
pd.DataFrame(logreg_bot_scaled).mean(axis=0)

fit_time       17.093319
score_time      0.056109
test_score      0.160243
train_score     0.160952
dtype: float64

**Logistic Regression, chars_wb + regex**

In [35]:
final = pd.concat([chars, df.drop(['text', 'From'], axis=1)], axis=1)
final.drop(final.columns[final.columns.duplicated()], axis=1, inplace=True)
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
final_ = StandardScaler().fit_transform(final_)
final_ = pd.DataFrame(final_, columns=final.columns)

In [40]:
reg = LogisticRegression(penalty='l1',
                         C=0.01,
                         multi_class='ovr', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42,
                         solver='liblinear')
logistic_reg_results = cross_validate(reg, final_, y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

In [39]:
pd.DataFrame(logistic_reg_results).mean(axis=0)

fit_time       145.164439
score_time       0.361391
test_score       0.704613
train_score      0.757955
dtype: float64

In [41]:
reg.fit(final_, y_train)
weights = list(zip(np.abs(reg.coef_).sum(axis=0), final_.columns))
weights.sort(key = lambda x: x[0], reverse=True)

C:\Users\ASUS\machine_learning_projector\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(


5000 feautres overfitted the model:

In [50]:
pd.DataFrame(chars_tfidf).mean(axis=0)

fit_time       13.519301
score_time      0.069105
test_score      0.836132
train_score     0.999735
dtype: float64

In [64]:
selected_columns = [x[1] for x in weights[:2000]]

In [67]:
reg = LogisticRegression(penalty='l2',
                         C=0.00005,
                         multi_class='multinomial', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42)
chars_tfidf = cross_validate(reg, final_[selected_columns], y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

In [66]:
pd.DataFrame(chars_tfidf).mean(axis=0)

fit_time       2.680824
score_time     0.020952
test_score     0.796005
train_score    0.853191
dtype: float64

**Logistic Regression, words + regex**

In [17]:
final = pd.concat([words, df.drop(['text', 'From'], axis=1)], axis=1)
final.drop(final.columns[final.columns.duplicated()], axis=1, inplace=True)
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
final_ = StandardScaler().fit_transform(final_)
final_ = pd.DataFrame(final_, columns=final.columns)

In [23]:
reg = LogisticRegression(penalty='l1',
                         C=0.01,
                         multi_class='ovr', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42,
                         solver='liblinear')
logistic_reg_results = cross_validate(reg, final_, y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

In [24]:
pd.DataFrame(logistic_reg_results).mean(axis=0)

fit_time       51.055507
score_time      0.109645
test_score      0.790613
train_score     0.840993
dtype: float64

In [25]:
reg.fit(final_, y_train)
weights = list(zip(np.abs(reg.coef_).sum(axis=0), final_.columns))
weights.sort(key = lambda x: x[0], reverse=True)

In [32]:
selected_columns = [x[1] for x in weights[:2000]]

In [38]:
reg = LogisticRegression(penalty='l2',
                         C=0.0000005,
                         multi_class='multinomial', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42)
words_tfidf = cross_validate(reg, final_[selected_columns], y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

In [40]:
pd.DataFrame(words_tfidf).mean(axis=0)

fit_time       1.550950
score_time     0.020808
test_score     0.833657
train_score    0.889296
dtype: float64

**Logistic Regression, chars + regex**

In [41]:
vectorizer = TfidfVectorizer(
                    analyzer='char',
                    ngram_range=(4,9),
                    vocabulary=None,
                    max_df=400, #0.8 
                    min_df = 200, #50
                    max_features=None,
                    smooth_idf=False,
                    norm='l2',
                    preprocessor=letters_only)
bag = vectorizer.fit_transform(df['text'])
_chars_ = pd.DataFrame(bag.astype(np.float32).todense(), columns = vectorizer.get_feature_names_out())

In [43]:
_chars_.head(1)

,abad,abas,abel,abet,abett,abette,abetter,abig,abilityt,ablean,...,zationin,zationl,zationne,zationo,zationr,zationst,zationw,zens,zethe,zine
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
final = pd.concat([_chars_, df.drop(['text', 'From'], axis=1)], axis=1)
final.drop(final.columns[final.columns.duplicated()], axis=1, inplace=True)
imputer = SimpleImputer(strategy='mean')
final_ = imputer.fit_transform(final)
final_ = StandardScaler().fit_transform(final_)
final_ = pd.DataFrame(final_, columns=final.columns)

In [47]:
reg = LogisticRegression(penalty='l1',
                         C=0.01,
                         multi_class='ovr', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42,
                         solver='liblinear')
logistic_reg_results = cross_validate(reg, final_, y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

In [48]:
pd.DataFrame(logistic_reg_results).mean(axis=0)

fit_time       144.440739
score_time       0.264891
test_score       0.704613
train_score      0.757822
dtype: float64

In [49]:
reg.fit(final_, y_train)
weights = list(zip(np.abs(reg.coef_).sum(axis=0), final_.columns))
weights.sort(key = lambda x: x[0], reverse=True)

C:\Users\ASUS\machine_learning_projector\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(


In [63]:
selected_columns = [x[1] for x in weights[:2000]]

In [67]:
reg = LogisticRegression(penalty='l2',
                         C=0.00007,
                         multi_class='multinomial', #multinomial, ovr
                         class_weight='balanced',
                         n_jobs=4,
                         random_state=42)
_chars_tfidf = cross_validate(reg, final_[selected_columns], y_train, cv=fixed_skf, scoring='accuracy',verbose=3, return_train_score=True)

In [66]:
pd.DataFrame(_chars_tfidf).mean(axis=0)

fit_time       2.513647
score_time     0.025955
test_score     0.801396
train_score    0.861145
dtype: float64